<div style="border-radius:10px; border:#4E5672 solid; padding: 15px; background-color: #F8F1E8; font-size:120%; text-align:left">

<h3 align="left"><font color='#4E5672'>📝 Description:</font></h3>


* An e-commerce company wants to segment its customers and determine marketing strategies based on these segments.
    
**Variables**
    
* InvoiceNo: Invoice number. A unique number assigned to each transaction or invoice. If it starts with 'C', it indicates a canceled transaction.
* StockCode: Product code. A unique number assigned to each product.
* Description: Product name.
* Quantity: Quantity of the product. Indicates how many units of the product were sold per invoice.
* InvoiceDate: Date and time of the invoice.
* UnitPrice: Price of the product (in Sterling).
* CustomerID: Unique customer number.
* Country: Country name. The country where the customer resides.
    

# Import Libraries 

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',None)
pd.set_option("display.float_format",lambda x : '%.3f'%x)

# Data Understanding

In [2]:
df_ = pd.read_csv("/kaggle/input/online-retail-customer-clustering/OnlineRetail.csv",sep=",", encoding="ISO-8859-1", header=0)
df = df_.copy()

In [3]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.550,17850.000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.390,17850.000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.750,17850.000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.390,17850.000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.390,17850.000,United Kingdom


In [4]:
def check_dataframe(df):
    print("######## SHAPE ########")
    print(df.shape)
    print("######## ISNULL ########")
    print(df.isnull().sum())
    print("######## INFO ########")
    print(df.info())
check_dataframe(df)

######## SHAPE ########
(541909, 8)
######## ISNULL ########
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64
######## INFO ########
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB
None


In [5]:
# What is the number of unique products?
df["Description"].nunique()

4223

In [6]:
# We have obtained the information on how many units of each product have been sold.
df["Description"].value_counts().head()

Description
WHITE HANGING HEART T-LIGHT HOLDER    2369
REGENCY CAKESTAND 3 TIER              2200
JUMBO BAG RED RETROSPOT               2159
PARTY BUNTING                         1727
LUNCH BAG RED RETROSPOT               1638
Name: count, dtype: int64

In [7]:
# We have calculated which product is the most ordered.
df.groupby("Description").agg({"Quantity":"sum"}).head()

,Quantity
Description,
4 PURPLE FLOCK DINNER CANDLES,144
50'S CHRISTMAS GIFT BAG LARGE,1913
DOLLY GIRL BEAKER,2448
I LOVE LONDON MINI BACKPACK,389
I LOVE LONDON MINI RUCKSACK,1


In [8]:
df.groupby("Description").agg({"Quantity":"sum"}).sort_values("Quantity",ascending=False).head()

,Quantity
Description,
WORLD WAR 2 GLIDERS ASSTD DESIGNS,53847
JUMBO BAG RED RETROSPOT,47363
ASSORTED COLOUR BIRD ORNAMENT,36381
POPCORN HOLDER,36334
PACK OF 72 RETROSPOT CAKE CASES,36039


In [9]:
# Calculate the TotalPrice.
df["TotalPrice"] = df["Quantity"] * df["UnitPrice"]


In [10]:
# We can calculate the total amount paid per invoice.
df.groupby("InvoiceNo").agg({"TotalPrice":"sum"}).head()


,TotalPrice
InvoiceNo,
536365,139.120
536366,22.200
536367,278.730
536368,70.050
536369,17.850


# Data Preparation

In [11]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
TotalPrice          0
dtype: int64

In [12]:
df.dropna(inplace=True)
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,406829.000,12.061,248.693,-80995.000,2.000,5.000,12.000,80995.000
UnitPrice,406829.000,3.460,69.315,0.000,1.250,1.950,3.750,38970.000
CustomerID,406829.000,15287.691,1713.600,12346.000,13953.000,15152.000,16791.000,18287.000
TotalPrice,406829.000,20.402,427.592,-168469.600,4.200,11.100,19.500,168469.600


In [13]:
# Returns are represented by the letter "C", and we need to exclude them from the dataset.
df = df[~df["InvoiceNo"].str.contains("C",na=False)]

# Calculating RFM Metrics Recency, Frequency, Monetary

In [14]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.550,17850.000,United Kingdom,15.300
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.390,17850.000,United Kingdom,20.340
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.750,17850.000,United Kingdom,22.000
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.390,17850.000,United Kingdom,20.340
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.390,17850.000,United Kingdom,20.340


In [15]:
df["InvoiceDate"].max()

'31-10-2011 17:13'

<div style="border-radius:10px; border:#4E5672 solid; padding: 15px; background-color: #F8F1E8; font-size:120%; text-align:left">

<h3 align="left"><font color='#4E5672'>📝 Note:</font></h3>
    
* We add 2 days to the date of the dataset and consider this date as the date of analysis.

In [16]:
today_date = dt.datetime(2011,11,2)
today_date

datetime.datetime(2011, 11, 2, 0, 0)

In [17]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format='%d-%m-%Y %H:%M')


In [18]:
rfm = (df.groupby('CustomerID').
       agg({'InvoiceDate': lambda InvoiceDate: (today_date - InvoiceDate.max()).days,
                                     'InvoiceNo': lambda InvoiceNo: InvoiceNo.nunique(),
                                     'TotalPrice': lambda TotalPrice: TotalPrice.sum()}))

In [19]:
rfm.head()

,InvoiceDate,InvoiceNo,TotalPrice
CustomerID,,,
12346.000,287,1,77183.600
12347.000,-36,7,4310.000
12348.000,37,4,1797.240
12349.000,-20,1,1757.550
12350.000,272,1,334.400


In [20]:
rfm.columns=['recency','frequency','monetary']


In [21]:
rfm.head()

,recency,frequency,monetary
CustomerID,,,
12346.000,287,1,77183.600
12347.000,-36,7,4310.000
12348.000,37,4,1797.240
12349.000,-20,1,1757.550
12350.000,272,1,334.400


In [22]:
rfm=rfm[rfm["monetary"]>0]

# Calculating RFM Scores

In [23]:
rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
rfm["monetary_score"] = pd.qcut(rfm["monetary"],5,labels=[1,2,3,4,5])
rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str)+
                    rfm['frequency_score'].astype(str))
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
CustomerID,,,,,,,
12346.000,287,1,77183.600,1,1,5,11
12347.000,-36,7,4310.000,5,5,5,55
12348.000,37,4,1797.240,2,4,4,24
12349.000,-20,1,1757.550,4,1,4,41
12350.000,272,1,334.400,1,1,2,11


# Creating & Analysing RFM Segments

In [24]:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}



rfm["segment"] = rfm['RFM_SCORE'].replace(seg_map,regex=True)
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
CustomerID,,,,,,,,
12346.000,287,1,77183.600,1,1,5,11,hibernating
12347.000,-36,7,4310.000,5,5,5,55,champions
12348.000,37,4,1797.240,2,4,4,24,at_Risk
12349.000,-20,1,1757.550,4,1,4,41,promising
12350.000,272,1,334.400,1,1,2,11,hibernating


In [25]:
rfm[["segment", "recency", "frequency", "monetary"]].groupby("segment").agg(["mean", "count"])

recency       frequency       monetary      
                       mean count      mean count     mean count
segment                                                         
about_to_sleep       14.312   352     1.162   352  471.994   352
at_Risk             114.786   593     2.879   593 1084.535   593
cant_loose           93.968    63     8.381    63 2796.156    63
champions           -32.638   633    12.417   633 6857.935   633
hibernating         178.605  1071     1.102  1071  488.643  1071
loyal_customers      -5.392   819     6.480   819 2864.248   819
need_attention       13.428   187     2.326   187  897.628   187
new_customers       -31.571    42     1.000    42  388.213    42
potential_loyalists -21.601   484     2.010   484 1041.222   484
promising           -15.489    94     1.000    94  294.008    94

In [26]:
rfm.loc[rfm["segment"]=="need_attention"].head(5)

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
CustomerID,,,,,,,,
12360.000,14,3,2662.060,3,3,5,33,need_attention
12372.000,33,3,1298.040,3,3,4,33,need_attention
12413.000,28,3,758.100,3,3,3,33,need_attention
12456.000,6,3,3181.040,3,3,5,33,need_attention
12536.000,5,3,12601.830,3,3,5,33,need_attention


<div style="border-radius:10px; border:#4E5672 solid; padding: 15px; background-color: #F8F1E8; font-size:120%; text-align:left">

<h3 align="left"><font color='#4E5672'>📝 Note:</font></h3>
    
* I have selected the 'need_attention' group here, 
and we have converted the customer IDs into a CSV file. This way, a process can be initiated to prevent the loss of customers in this group.

In [27]:
need_attention_df = pd.DataFrame()

In [28]:
need_attention_df["need_attention_id"] = rfm[rfm["segment"]=="need_attention"].index
need_attention_df["need_attention_id"] = need_attention_df["need_attention_id"].astype(int)

In [29]:
need_attention_df.to_csv("need_attention.csv")